In [5]:
from qdrant_client import QdrantClient
client = QdrantClient(url="0.0.0.0")  # Persists changes to disk, fast prototyping

In [8]:
from typing import List

In [1]:
from torch import cuda

In [9]:
class VectorSearch:

    def __init__(self, collection_name: str, model: str, host: str, port: int):
        self.collection_name = collection_name
        
        device = 'cpu'
        if cuda.is_available():
            device = 'gpu'
        self.model = SentenceTransformer(model, device)
        
        self.client = QdrantClient(host, port)

    def search(self, query_string: str, filter_: dict = None) -> List[dict]:
        vector = self.model.encode(query_string).tolist()
        hits = self.qdrant_client.search(
            collection_name=self.collection_name,
            query_vector=vector,
            query_filter=Filter(**filter_) if filter_ else None,
            top=5
        )
        return [hit.payload for hit in hits]

In [11]:
import qdrant_client

In [16]:
from qdrant_client.models import VectorParams

In [13]:
client.recreate_collection("theses_metadata", "")

AttributeError: module 'qdrant_client' has no attribute 'create_collection'

In [20]:
import numpy as np
from nbconf import *

In [21]:
config.DATA_PATH

'/home/decarv/projects/sse/data'

In [28]:
ls ../data

clean_data.csv  documents/  models/                   vectors.npy
dataframe       errors.db   vectors_experiment_3.npy  webpages.db
dataframe.csv   metadata/   vectors_experiment_4.npy  your_database.db


In [24]:
import os

In [30]:
vectors = np.load(os.path.join(config.DATA_PATH, "vectors_experiment_4.npy"))

<module 'config' from '/home/decarv/projects/sse/sse/config.py'>

In [ ]:
with open(os.path.join(config.DATA_PATH, "idxs.pickle"), 'rb') as f:
    f.

In [33]:
vectors.shape

(1365328, 768)

In [35]:
vector_params = VectorParams(size=vectors.shape[1], distance="Cosine")

In [36]:
client.recreate_collection("theses_metadata", vectors_config=vector_params)

True

In [15]:
from typing import List
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from torch import cuda


class SearchClient:

    def __init__(self, collection_name: str, model: str, host: str, port: str):
        self.collection_name = collection_name

        device = 'cuda' if cuda.is_available() else 'cpu'
        self.model = SentenceTransformer(model, device=device)

        self.client = QdrantClient(host, port)

    def search(self, query_string: str, filter_: dict = None) -> List[dict]:
        vector = self.model.encode(query_string).tolist()
        hits = self.client.search(
            collection_name=self.collection_name,
            query_vector=vector,
            query_filter=Filter(**filter_) if filter_ else None,
            top=5
        )
        return [hit.payload for hit in hits]

In [20]:
from fastapi import FastAPI

import nbconf
import config


class Server:
    """"""

    def __init__(self, search_client: SearchClient):
        self.search_client = search_client

        self.app = FastAPI()

        @self.app.get("/search")
        def search_query(query_string: str):
            result = self.search_client.search(query_string)
            return {
                "result": result
            }
        self.search = search_query

In [21]:
COLLECTION_NAME = "theses"
MODEL_NAME = "distilbert-base-nli-stsb-mean-tokens"

HOST = "0.0.0.0"
PORT = "6333"

In [22]:
search_client = SearchClient(COLLECTION_NAME, MODEL_NAME, HOST, PORT)
server = Server(search_client)

In [23]:
import uvicorn
uvicorn.run(server.app, host="0.0.0.0", port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop